# Notebook 1: Processing FASTQ files using QIIME2 </br>

**Welcome to Ge/Bi/ESE 246!** In this first session, we'll give a brief overview on the theory behind 16S rRNA amplicon sequencing and its use in modern microbial ecology, using some work I've done in my graduate studies as a tutorial. We'll then dive in and look at the raw data produced by the sequencing platform, and proceeed to process these data using [QIIME2](https://docs.qiime2.org/2019.10/).

**16S rRNA sequencing reveals microbial diversity**

Advances in DNA sequencing have transformed microbial ecology by providing access to the genetic information of microorganisms that have thus far evaded laboratory culture. Sequencing of one particular gene–that coding for the 16S subunit of ribosomal RNA–has been of particular utility for identifying the taxonomic composition of microbial communities. The 16S ribosomal RNA (rRNA) gene (and its eukaryotic equivalent, the 18S rRNA gene) is useful as it is present in virtually all cellular organisms, as it provides critical structural support to the ribosome and binds incoming mRNA. As a result of the important role the 16S rRNA gene plays in translation, its sequence is highly-conserved across taxa, preventing the sequence from becoming 'saturated' with mutations which would produce spurious results upon comparison. Thus, sequencing of the 16S rRNA gene has been used to directly compare cellular organisms [across the tree of life](https://www-pnas-org.clsproxy.library.caltech.edu/content/74/11/5088), giving us a tool to quickly place any given 16S rRNA sequence within a near-universal phylogenetic framework. 

**Next-generation sequencing revolutionizes microbial ecology**

Prior to the advent of high-throughput sequencing platforms, such as [Illumina's MiSeq platform](https://www.youtube.com/watch?v=fCd6B5HRaZ8), the enterprising microbial ecologist was limited to using techniques such as [Sanger sequencing](https://www-pnas-org.clsproxy.library.caltech.edu/content/74/12/5463), capable of recovering a single sequence ('read') for each sample provided. Thus, studies looking at 16S sequences recovered from uncultured prokaryotes found in the environment focused on the diversity these sequences exhibited. Typically, these studies PCR-amplified the whole 16S rRNA gene (~1500 base pairs) using primers designed to bind universally to Bacteria and Archaea 16S. Limited to ~10$^2$ sequences, each study could only evaluate the structure or composition of different communities semi-quantitatively.

In contrast, 'next-generation' sequencing platforms like MiSeq can provide to 16S rRNA sequencing studies many (10$^5$-10$^6$) reads per sample, albeit at somewhat more limited phylogenetic resolution, as PCR amplification is typically performed to produce a 'library' of ~400 bp fragments ('amplicons') of the 16S gene that are then sequenced. While a shorter sequence provides less data used to place a given sequence into the tree of life, it can be easily sequenced on a next-generation platform.  The resulting data products can then be analyzed for far more robust analysis of  community structure and composition within a given sample and across habitats, using tools such as Jari Oksanen's famous R package, [vegan](https://cran.r-project.org/web/packages/vegan/vegan.pdf), which we'll use often later in the course.

**The project**

Now that we have the historical context, let's jump into looking at real data. For this tutorial, we'll be looking at samples I collected as a part of my work applying [heavy-liquids separation](https://en.wikipedia.org/wiki/Sodium_metatungstate) to [methane seep](https://en.wikipedia.org/wiki/Cold_seep) sediments. The idea here was to explore the potential utility of this technique to purify microbial communities that associate with minerals such as [pyrite](https://www.mindat.org/min-3314.html) and [magnetite](https://www.mindat.org/min-2538.html) that are often found in marine sediments. Conductive materials, including magnetite, have been shown by [Amelia-Elena Rotaru and coworkers](https://mbio.asm.org/content/9/3/e00226-18.full) to stimulate direct, interspecies electron transfer (DIET) from acetate-oxidizing Bacteria to methanogenic Archaea. In methane seep sediments, [multicellular aggregates of archaeal methanotrophs (ANME) and sulfate-reducing Bacteria (SRB)](https://science.sciencemag.org/content/293/5529/484.long) [are also thought to use DIET](https://www.nature.com/articles/nature15512) to instead perform [anaerobic methane oxidation (AOM)](https://www.annualreviews.org/doi/full/10.1146/annurev.micro.61.080706.093130?url_ver=Z39.88-2003&rfr_id=ori%3Arid%3Acrossref.org&rfr_dat=cr_pub%3Dpubmed). Given that conductive minerals including magnetite have been shown to stimulate DIET and the importance of DIET in AOM syntrophy, we hypothesized that certain ANME and/or SRB taxa may preferentially associate with magnetite or pyrite surfaces. We further hypothesized that we could  detect this association in 16S amplicon libraries prepared from the bulk sediment and a from dense, pyrite- and magnetite-rich fraction of methane seep sediments. 

Here, we'll start the process of exploring this hypothesis by processing the raw data from a MiSeq platform operated by [Laragen, Inc.](http://www.laragen.com/laragen_nextgen.php). Prior to sequencing, PCR amplification targeted a [411 base pair region](https://sfamjournals.onlinelibrary.wiley.com/doi/full/10.1111/1462-2920.13023) of the 16S rRNA gene from replicate samples of methane seep sediments separated by density using sodium metatungstate. After paired-end sequencing on a MiSeq, amplicon libraries each appear in separate [CASAVA 1.8](http://illumina.bioinfo.ucr.edu/ht/documentation/data-analysis-docs/CASAVA-FASTQ.pdf/view) [FASTQ files](https://support.illumina.com/bulletins/2016/04/fastq-files-explained.html). Each FASTQ file consists of a number of reads (individual sequences detected by the sequencing platform) annotated by the quality of each base-pair call within each sequence. Let's take a look:

In [1]:
source activate qiime2-2019.10

(qiime2-2019.10) 


In [2]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /Users/kylemetcalfe/git/OrphanLabTag/Data \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

Imported /Users/kylemetcalfe/git/OrphanLabTag/Data as CasavaOneEightSingleLanePerSampleDirFmt to demux-paired-end.qza
(qiime2-2019.10) 


In [3]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --p-trunc-len-f 240 \
  --p-trunc-len-r 200 \
  --p-n-threads 0 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoising-stats.qza
(qiime2-2019.10) 


In [5]:
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

Saved Visualization to: rep-seqs.qzv
(qiime2-2019.10) 


We'll use the [SILVA](https://www.arb-silva.de/documentation/release-132) database of aligned 16S sequences as a taxonomic framework, but it is important to note that there are [other frameworks](https://www.nature.com/articles/nbt.4229) that use a set of many conserved genes in addition to 16S to assign taxonomy.

In [6]:
qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

Saved Visualization to: denoising-stats.qzv
(qiime2-2019.10) 
